In [1]:
import pandas as pd

In [6]:
data = pd.read_csv("review_final_38.csv", encoding="cp949")
del data["Unnamed: 0"]

In [7]:
import re
data["reple"] = [i.replace("BEST","") for i in data["reple"]]
def token_word(line):    
    hangul = re.compile('[^ㄱ-ㅣ가-힣]+') # 한글과 띄어쓰기를 제외한 모든 글자
    # hangul = re.compile('[^ \u3131-\u3163\uac00-\ud7a3]+')  # 위와 동일
    result = hangul.sub(' ', line) # 한글과 띄어쓰기를 제외한 모든 부분을 제거
    
    return result

In [8]:
reple = [k for k in data["reple"]]
reple = [token_word(k) for k in reple]

In [91]:
reple[0]

'강동원 너 나오면 꼭 보러가는거 알지 최소 다섯번 '

In [9]:
from konlpy.tag import Twitter
twitter = Twitter()
token = [twitter.morphs(i) for i in reple]

In [28]:
all_token = [j for i in token for j in i if len(j)>=2]

In [157]:
all_token = pd.Series(all_token)
vocab = all_token.value_counts()

In [158]:
vocab = vocab.index
vocab_list = [i for i in vocab]

In [159]:
voca = vocab_list[:10000] # 욕하는(982) [40000]

In [77]:
from sklearn.feature_extraction.text import CountVectorizer

In [160]:
vect = CountVectorizer(min_df=100).fit(reple)

In [161]:
vo = vect.vocabulary_.keys()
len(vo)

936

In [162]:
vector = vect.transform(reple[:10000]).toarray()

In [163]:
X = vector

In [164]:
X = pd.DataFrame(X, columns=vo)

In [166]:
X.shape

(10000, 936)

In [167]:
y = data["y"][:10000]

In [168]:
from sklearn.cross_validation import train_test_split # 버전에 따라 다름

train_X, test_X,train_Y,test_Y = train_test_split(X, y, random_state = 0)

In [184]:
from sklearn.linear_model import RidgeCV, LassoCV, ElasticNetCV
lasso = LassoCV(alphas = [ 0.002,0.004,0.005, 0.007,
                          0.0001, 0.0003, 0.0006, 0.001, 0.00003, 0.00006, 1,2,3,4])
lasso.fit(train_X, train_Y)
alpha = lasso.alpha_
print("Best alpha : ", alpha)

lasso = LassoCV(alphas = [alpha *.6, alpha * .65, alpha * .7,alpha * .75,
                         alpha * .8, alpha * .85, alpha * .9, alpha * .95,
                         alpha * 1.05, alpha * 1.1, alpha * 1.15, alpha * 1.25,
                         alpha * 1.3, alpha * 1.35, alpha * 1.4],
               max_iter = 50000, cv=10)
lasso.fit(train_X, train_Y)
alpha = lasso.alpha_
print("Best alpha : ", alpha)
pred_lasso = lasso.predict(test_X)
#print('Accuracy: %.2f' % accuracy_score(test_Y, pred_lasso))

coefs = pd.Series(lasso.coef_, index = train_X.columns)
print("lasso는 변수를 몇개 선택했니? : ", sum(coefs!=0)) 
print("lasso는 변수를 몇개 버렸니? : ",sum(coefs==0)) 


Best alpha :  0.0003
Best alpha :  0.000315
lasso는 변수를 몇개 선택했니? :  247
lasso는 변수를 몇개 버렸니? :  689


In [173]:
coef = pd.DataFrame(lasso.coef_, columns=["value"])

In [176]:
coef[coef["value"] > 0] = 1
coef[coef["value"] < 0] = 0

In [179]:
coef = [i for i in coef["value"]]

In [197]:
xx = X
for i in range(len(X)):
    xx.loc[i] =xx.loc[i] *coef 

In [209]:
k=[]
for i in range(len(X)):
    k.append(sum(xx.loc[i]))

In [212]:
K = pd.DataFrame(k, columns=["1"])

In [220]:
positive_index = K[K["1"] > 1].index

In [233]:
sample.drop(positive_index)

,0
0,강동원 너 나오면 꼭 보러가는거 알지 최소 다섯번
2,황정민 이번작품도 기대하겠습니다
3,황정민 이번 영화 다들 장난 아닌듯
4,동원오빠 훌륭한연기 기대할게요 응원합니다
5,와아아아 강동원 나와라아아아아
8,정민이형 이번에도 기대
12,올해안에꼭개봉했으면ㅠㅠ강동원 남우주연상 후보가자
13,강 동 원 사랑해요
15,강동원꼭나와라라아아아
16,강동원이 나오면 그냥 닥치고 봐야한다


In [232]:
sample1 = reple[:10000]
sample = pd.DataFrame(sample1)
sample.loc[positive_index]

,0
1,강동원 황정민 이성민 티켓파워는 걱정없고 영화만 잘 빠지면 백퍼 흥행한다
6,강동원 기대할게요
7,여기 강동원 나오나요 강동원 보고싶다ㅜㅜ
9,황정민 강동원 걍 지릴듯
10,황정민 강동원 후덜덜 짱 재미있을 듯
11,강동원 꼭 나와라 황정민 강동원 조합 보고싶다
14,강동원 쉬기는 하는거임ㅠ 작품 진짜 많이 한다
21,강동원 짝 강동원 짝
27,말이필요없는 조합 황정민 강동원
28,황정민 강동원 박성웅 무조건 위


In [14]:
from gensim.models.word2vec import Word2Vec
model = Word2Vec(text_train ,min_count=12, sg=1) # skip-gram

In [15]:
model.init_sims(replace=True)

In [16]:
w2v = dict(zip(model.wv.index2word, model.wv.syn0))